<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 3.0 Pretraining Language Models

"off-the-shelf(바로)"으로 사용할 수 있는 많은 사전 훈련된 BERT 모델들이 있습니다.  그러나 다운스트림 NLP 작업을 위해 새로운 언어 모델을 학습하거나 미세 조정 (Fine tuning)하는 것이 유리한 경우가 있습니다.  예를 들어, 의학 논문은 의학 분야에 특화된 어휘를 사용하기 때문에, 의학 논문으로 학습된 언어 모델은 일반 텍스트로 훈련받은 언어 모델보다 의학 텍스트를 처리하는 프로젝트에 더 적합합니다.  

이번 노트북에서는 도메인별 데이터를 사용하여 BERT 언어 모델을 사전 훈련하는 방법에 대해 설명합니다.
    
**[3.1 Data Preparation](#3.1-Data-Preparation)<br>**
**[3.2 Training the BERT Tokenizer](#3.2-Training-the-BERT-Tokenizer)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[3.2.1 Exercise: Tokenize a Term](#3.2.1-Exercise:-Tokenize-a-Term)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2.2 Update the BERT Vocabulary](#3.2.2-Update-the-BERT-Vocabulary)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2.3 Exercise: Train a Larger Vocabulary](#3.2.3-Exercise:-Train-a-Larger-Vocabulary)<br>
**[3.3 Launch BERT Pretraining with NeMo](#3.3-Launch-BERT-Pretraining-with-NeMo)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[3.3.1 TensorBoard Visualization](#3.3.1-TensorBoard-Visualization)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.3.2 Practical Considerations](#3.3.2-Practical-Considerations)<br>

---
# 3.1 Data Preparation

BERT와 같은 마스킹된 뉴럴 언어 모델은 텍스트에 대해 학습됩니다. 그러나 먼저 토큰화를 통해 텍스트가 숫자 표현으로 변환되는 프로세스를 진행해야 합니다.  네트워크는 입력 문장에서 임의의 단어를 마스킹하고 누락된 단어를 예측함으로써 훈련됩니다. 그런 다음 학습된 언어 모델을 다운스트림 NLP 작업에 사용할 수 있으며, 여기서 이를 "사전 훈련된(Pretrained)" 언어 모델이라고 합니다.

NVIDIA NeMo를 사용하면 토큰화를 트레이닝 중에 즉각적으로(on-the-fly) 진행하거나 트레이닝 전에 오프라인으로 수행할 수 있습니다.

- **온더플라이 데이터 전처리(On-the-fly data preprocessing):** 트레이닝 및 검증 텍스트 파일에는 공백(space)으로 구분된 단어가 있어야 합니다:
                                [WORD] [SPACE] [WORD] [SPACE] [WORD] [SPACE] [WORD]

                                
- **오프라인 데이터 전처리(Offline data preprocessing):** 데이터는 HD5F 형식으로 미리 준비됩니다. 이는 큰 텍스트 말뭉치(corpora)에 권장되는 사전 처리 방식입니다.  오프라인 데이터 전처리 스크립트는 [BERT quick start guide](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/LanguageModeling/BERT#quick-start-guide) 를 참조하십시오. 

이번 예시에서는 온더플라이 데이터 전처리 파이프라인을 사용합니다.  [NCBI-disease corpus](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/) 에 대해 BERT를 교육할 것입니다.
NCBI 말뭉치(corpus)는 793개의 PubMed 요약본 세트입니다.  우리의 목표는 의료 분야를 위한 사전 검증된 모델을 만드는 것입니다.  다음은 텍스트 요약의 예입니다.:

In [ ]:
! tail -5 /dli/task/data/train.txt

---
# 3.2 Training the BERT Tokenizer

이전 노트북에서 설명한 것처럼 BERT 토크나이저는 텍스트를 미리 정의된 어휘 다음에 있는 토큰으로 분할합니다. 토큰화 알고리즘은 텍스트 말뭉치에서 상위 K개로 자주 사용되는 단어의 변형(variant)에 이은 어휘를 생성합니다.

어휘의 크기에 따라 트레이닝 코스트가 증가하기 때문에 어휘 크기가 제한됩니다. 텍스트 말뭉치의 모든 고유한 단어를 어휘에 포함시키는 것은 토크나이저의 능력을 넘어 트레이닝의 복잡성을 폭발시킬 것입니다. 예를 들어, 2018년에 출시된 BERT 모델에는 WordPiece라는 서브워드 토큰화 알고리즘이 포함되어 있으며, 어휘 제한은 30,000개입니다.

그렇다면 토크나이저는 어휘에 포함되지 않은 용어 또는 **out-of-vocabulary (OOV)** 단어를 어떻게 다룰까요?

1. 한 가지 옵션은 OOV 단어를 특수 토큰 \[UNK\]으로 바꾸는 것입니다. 이 경우, 모든 OOV 용어는 의미(semantic)를 상실하는 신경망에 대해 동일한 표현을 가질 것입니다. 
2. 두 번째 옵션은 OOV 단어를 문자 수준에서 분할하는 것입니다. 이것은 뉴럴 언어 모델에 대한 입력의 크기를 증가시키고, 의미(시멘틱)를 유지하기 위해 문자 사이의 관계를 학습하는 과제를 더합니다.
3. BERT WordPiece와 같은 서브워드 토큰은 단어 토큰과 문자 분할 옵션 사이에 솔루션을 제공합니다. OOV 단어를 서브워드로 토큰화합니다.

이제 `bert-base-uncased` 토크나이저를 살펴보겠습니다.:

In [ ]:
# import nemo nlp collection 
from nemo.collections import nlp as nemo_nlp

# load the bert-base-uncased tokenizer 
tokenizer_uncased = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-uncased")

In [ ]:
# get the vocabulary size
print(" The vocabulary size: ", tokenizer_uncased.vocab_size)

예를 들어, 몇 년 동안 BERT를 사용한 형식 토큰화(format tokenization)를 살펴보십시오. 2021년 이전 연도 정보는 어휘의 일부가 될 만큼 말뭉치에 자주 나타나지만, 미래의 연도 정보는 OOV이고 서브 토큰으로 분할됩니다.

아래 셀에서 `tokenizer_uncased.text_to_tokens()`를 사용하여 시도해 보십시오.

In [ ]:
# Bert tokenizer for years
print("Tokenized year: ", tokenizer_uncased.text_to_tokens('2019'))
print("Tokenized year: ", tokenizer_uncased.text_to_tokens('2020'))
print("Tokenized year: ", tokenizer_uncased.text_to_tokens('2021'))
print("Tokenized year: ", tokenizer_uncased.text_to_tokens('2022'))
print("Tokenized year: ", tokenizer_uncased.text_to_tokens('2023'))
print("Tokenized year: ", tokenizer_uncased.text_to_tokens('2030'))

연도 토큰화 예는 프로세스에 대한 직관을 제공합니다.  의학 전문 용어와 같은 영역별 컨텍스트는 어떻습니까? 구체적인 예를 들어, 다음 문장으로 다시 시도하십시오.

_"더 많은 연구에 따르면 C5D 혈청의 낮은 농도의 희석액은 C5 억제제나 불활성화제가 아닌 C5 용혈 분석의 일부 단계에서 간섭 요인이 포함되어 있습니다. / Further studies suggested that low dilutions of C5D serum contain a factor or factors interfering at some step in the hemolytic assay of C5 rather than a true C5 inhibitor or inactivator."_

이 문장은 dilutions, C5D, C5, hemolytic 및 assay과 같은 여러 의학 용어를 포함합니다.

In [ ]:
# Bert tokenizer for domain-specific example
SAMPLES = "Further studies suggested that low dilutions of C5D serum contain a factor or factors interfering at some step in the hemolytic assay of C5 rather than a true C5 inhibitor."
print("Tokenized sentence: ", tokenizer_uncased.text_to_tokens(SAMPLES))

서브워드로 토큰화된 의학 전문 용어를 볼 수 있습니다. : 
- dilutions -> 'dil', '##ution', '##s'
- hemolytic ->'hem', '##ol', '##ytic'
- assay -> 'ass', '##ay'
- C5 ->'c', '##5'
- C5D ->'c', '##5', '##d'

dilutions, hemolytic 및 assay 과 같은 의학 용어는 표준 BERT 토크나이저 어휘에는 없습니다. 따라서 개별적으로 토큰화할 수 없으며 서브워드로 나뉩니다.

## 3.2.1 Exercise: Tokenize a Term
아래의 "FIXME" 행을 수정하여 BERT 토크나이저를 사용하여 "COVID-19" 용어를 토큰화하십시오.  필요한 경우 [솔루션](solutions/ex3.2.1.ipynb) 을 확인하십시오.

In [ ]:
# Tokenize a new term
TEXT = "" #FIXME
print("Tokenized sentence: ") #FIXME

## 3.2.2 Update the BERT Vocabulary

`tokenizer_uncased.tokenizer.add_tokens()` 함수를 사용하여 도메인별 단어를 토크나이저 어휘에 추가할 수 있습니다. 각 새 토큰에 대한 임베딩 벡터는 임의의 값으로 초기화됩니다.

In [ ]:
# Add some medical jargon to the vocabulary of Bert tokenizer
additional_tokens = tokenizer_uncased.tokenizer.add_tokens(["dilutions", "hemolytic"])
print(" The vocabulary size before: ", tokenizer_uncased.vocab_size)
print(" The vocabulary size after : ", tokenizer_uncased.vocab_size)

In [ ]:
# Tokenize the sentence with the new vocabulary 
print("Tokenized sentence: ", tokenizer_uncased.text_to_tokens(SAMPLES))

어휘에 포함할 도메인별 단어 수가 많을 경우, 사전 훈련된 토크나이저를 사용하는 대신 도메인별 말뭉치에서 새 토크나이저를 트레이닝하는 것이 가장 좋습니다. 

[NCBI-disease corpus] 말뭉치에 대한 새로운 WordPiece 토크나이저를 교육하여 어휘 크기를 10,000개로 제한합니다.

In [ ]:
vocab_size= 10000
text_corpus=["/dli/task/data/train.txt"]

# add the special tokens required for BERT pretraining.
special_tokens = ["<PAD>","<UNK>","<CLS>","<SEP>","<MASK>"]

In [ ]:
from tokenizers import BertWordPieceTokenizer

my_bert_tokenizer = BertWordPieceTokenizer()
my_bert_tokenizer.train(files=text_corpus, vocab_size=vocab_size,
                        min_frequency=1, special_tokens=special_tokens,
                        show_progress=True, wordpieces_prefix="##")

In [ ]:
# get the new vocabulary size
print(" The new vocabulary size  : ", len(my_bert_tokenizer.get_vocab()))

In [ ]:
# save the new vocabulary 
my_bert_tokenizer.save_model(directory="/dli/task/data/")

In [ ]:
!tail -20 /dli/task/data/vocab.txt 

어휘가 정의되면 `nemo_nlp.modules.get_tokenizer()`함수를 사용하여 토크나이저를 새 어휘로 불러올 수 있습니다. 이전 텍스트 샘플들을 토큰화하고 바닐라 BERT 토크나이저(vanilla BERT tokenizer)와 비교해 보겠습니다. 
이제 도메인별 전문 용어를 개별 토큰들로 인코딩해야 합니다.

In [ ]:
# load the tokenizer from the vocabulary 
special_tokens_dict = {"unk_token": "<UNK>", "sep_token": "<SEP>", "pad_token": "<PAD>", "bos_token": "<CLS>", "mask_token": "<MASK>","eos_token": "<SEP>", "cls_token": "<CLS>"}
tokenizer_custom = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-uncased", vocab_file='/dli/task/data/vocab.txt', special_tokens=special_tokens_dict)

print("BERT tokenizer with custom vocabulary: ", tokenizer_custom.text_to_tokens(SAMPLES))

## 3.2.3 Exercise: Train a Larger Vocabulary 

어휘 크기가 15,000개인 BERT 토크나이저를 트레이닝하려면 "FIXME" 행을 수정하십시오. 필요한 경우 [솔루션](solutions/ex3.2.3.ipynb)을 확인하십시오.

In [ ]:
# Train a larger vocabulary 
vocab_size = None #FIXME
my_bert_tokenizer_15k= None #FIXME
my_bert_tokenizer_15k.train(files=text_corpus, vocab_size=vocab_size, 
                            min_frequency=1, special_tokens=special_tokens, 
                            show_progress=True, wordpieces_prefix="##")
print(" The new vocabulary size  : ", len(my_bert_tokenizer_15k.get_vocab()))

---
# 3.3 Launch BERT Pretraining with NeMo

[bert_pretraining_from_text_config.yaml](nemo/examples/nlp/language_modeling/conf/bert_pretraining_from_text_config.yaml) 와 트레이닝 스크립트인 [bert_pretraining.py](nemo/examples/nlp/language_modeling/bert_pretraining.py) 를 사용하여 온더플라이(on-the-fly, 즉각적인) 데이터 전처리를 위한 모델 구성을 사용합니다. YAML 구성 파일은 트레이닝 스크립트에 필요한 매개변수를 제공하며, 필요에 따라 매개변수 값을 재정의할 수 있습니다. 

NeMo 구성 파일 및 스크립트에 대한 자세한 내용은 뒷부분 모듈에서 확인할 수 있습니다.  일단 구성 파일에 몇 가지 중요한 YAML 키만 적어두겠습니다.
- `trainer` : GPU 수, 혼합 정밀도 훈련, 에포크 수 등의 트레이닝 프로세스 파라미터
- `model.only_mlm_loss`: 다음 문장 예측 없이 마스킹된 언어 모델을 사용
-`model.mask_prob`: 데이터 처리 중에 입력 텍스트에서 토큰을 마스킹할 확률
- `model.train_ds`/`model.validation_ds`: 데이터 집합 매개 변수
- `model.tokenizer`: 토크나이저 매개 변수
- `model.language_model`: 언어 모델 아키텍처 매개 변수
-`model.optim`: 최적화 도구 매개 변수

bert_pretraining 매개 변수에 대한 자세한 내용은 [NeMo documentation](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/bert_pretraining.html#quick-start-guide)을 참조하십시오.

전처리된 데이터가 있는 BERT 오프라인 사전 훈련의 경우 전용 구성인 [bert_pretraining_from_preprocessed_config.yaml](nemo/examples/nlp/language_modeling/conf/bert_pretraining_from_preprocessed_config.yaml) 파일을 사용합니다.

In [ ]:
# Show the configuration file
! cat nemo/examples/nlp/language_modeling/conf/bert_pretraining_from_text_config.yaml

In [ ]:
%%time
# Override the parameters specific to our data; run only two epochs for now
! python nemo/examples/nlp/language_modeling/bert_pretraining.py \
    model.train_ds.data_file=/dli/task/data/train.txt\
    model.validation_ds.data_file=/dli/task/data/test.txt\
    model.tokenizer.vocab_file=/dli/task/data/vocab.txt\
    model.train_ds.batch_size=16 \
    trainer.max_epochs=2

## 3.3.1 TensorBoard Visualization
다음 TensorBoard  링크를 클릭하여 nemo_experiments 폴더에 저장된 로스와 정확도와 같은 실험적인 메트릭에 대한 그래프를 살펴봅니다.<br>
[TensorBoard](/tensorboard/)

## 3.3.2 Practical Considerations

트랜스포머 기반 언어 모델을 사전 훈련하는 것은 레이블링된 텍스트 코퍼스 데이터셋을 요구하기 않습니다. 하지만, 많은 양의 데이터와 연산 시간을 필요로 합니다. 예를 들어 [English Wikipedia](https://huggingface.co/datasets/wikipedia) 과 [bookcorpus](https://huggingface.co/datasets/bookcorpus)를 사용하여 8개의 V100 GPU 가 있는 NVIDIA DGX-1 서버에서 BERT 모델을 사전 훈련하는 경우 혼합 정밀도 모드에서 6일이 걸립니다. BERT 트레이닝과 파인 튜닝 성능에 대해서는 아래 페이지에서 더 확인해보실 수 있습니다. - https://catalog.ngc.nvidia.com/orgs/nvidia/resources/bert_for_pytorch/performance.

반면에, 트랜스포머 기반의 모델을 파인 튜닝하는 것은 상대적으로 연산 집약적이지 않지만 레이블링된 데이터를 요구합니다. 파트 2에 있는 실습에서는 BERT 모델을 파인 튜닝하는 방법에 집중하여 텍스트 분류 및 명명된 개체 인식(NER)과 같은 다운스트림 NLP 태스크에 대해 배워 봅니다.

---
<h2 style="color:green;">축하합니다.!</h2>

BERT 사전훈련 노트북을 완료하였습니다!  

여러분은 다음을 학습하였습니다:
* BERT 토크나이저로 학습하기
* NeMo로 BERT 언어 모델을 사전훈련하기

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>